In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [22]:
seed = 1
batch_size = 64
test_batch_size = 64

no_cuda = False

In [23]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [24]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [26]:
model = Net().to(device)

In [27]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

In [28]:
params = list(model.parameters())
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


In [29]:
model.train()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [30]:
data, target = next(iter(train_loader))

In [31]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [32]:
data, target = data.to(device), target.to(device)

In [33]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [34]:
optimizer.zero_grad()

In [35]:
output = model(data)

In [36]:
loss = F.nll_loss(output, target)

In [37]:
loss.backward()

In [38]:
optimizer.step()

In [39]:
epochs = 1
log_interval = 100

In [40]:
for epoch in range(1, epochs+1):
   
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0 :
            print('Train Epoch: {} [ {}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()
            ))
    
   

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader : 
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)))

Train Epoch: 1 [ 0/60000 (0%)]	Loss: 2.320928
Train Epoch: 1 [ 6400/60000 (11%)]	Loss: 2.232695
Train Epoch: 1 [ 12800/60000 (21%)]	Loss: 2.129740
Train Epoch: 1 [ 19200/60000 (32%)]	Loss: 1.999498
Train Epoch: 1 [ 25600/60000 (43%)]	Loss: 1.665831
Train Epoch: 1 [ 32000/60000 (53%)]	Loss: 1.361006
Train Epoch: 1 [ 38400/60000 (64%)]	Loss: 0.853679
Train Epoch: 1 [ 44800/60000 (75%)]	Loss: 0.650575
Train Epoch: 1 [ 51200/60000 (85%)]	Loss: 0.641776
Train Epoch: 1 [ 57600/60000 (96%)]	Loss: 0.798732
Test set: Average Loss: 0.4794, Accuracy: 8658/10000 ( 87%)

